In [4]:
import requests
import pandas as pd
import time

# 카카오 API 키
KAKAO_API_KEY = "34a6059d6b30f92770b7805eeef48c9a"  # 자신의 API 키로 변경

# 데이터 불러오기
file_path = "../data/cleaned_jobs.csv"  # 데이터 파일 경로
df = pd.read_csv(file_path)

# 구인 중인 데이터 필터링
df = df[df["구인상태"] == "구인중"]

# 중복 제거 (동일한 회사명에 대해 API 호출 중복 방지)
unique_companies = df["업체명"].drop_duplicates()

# 카카오 API 호출 함수
def get_location_from_kakao(company_name, max_retries=3):
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?query={company_name}"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    
    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                documents = response.json().get("documents", [])
                if documents:
                    first_result = documents[0]
                    # 응답 데이터에서 필요한 값 추출
                    address_name = first_result.get("address_name", None)  # 전체 주소
                    latitude = float(first_result["y"])  # 위도
                    longitude = float(first_result["x"])  # 경도
                    return address_name, latitude, longitude
                else:
                    print(f"검색 결과 없음: {company_name}")
                    return None, None, None
            else:
                print(f"API 호출 오류 {response.status_code}: {response.text}")
        except Exception as e:
            print(f"API 호출 시 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
            time.sleep(1)  # 잠시 대기 후 재시도
    return None, None, None

# 위치 정보 가져오기
company_to_location = {}
for company in unique_companies:
    address, lat, lon = get_location_from_kakao(company)
    company_to_location[company] = (address, lat, lon)
    time.sleep(0.1)  # API 호출 속도 제한

# 위치 정보를 데이터프레임에 추가
addresses = []
latitudes = []
longitudes = []

for company in df["업체명"]:
    address, lat, lon = company_to_location.get(company, (None, None, None))
    addresses.append(address)
    latitudes.append(lat)
    longitudes.append(lon)

df["주소"] = addresses
df["위도"] = latitudes
df["경도"] = longitudes

# 유효한 데이터만 필터링
df = df.dropna(subset=["위도", "경도"])

# 결과 출력 및 저장
output_path = "../data/cleaned_jobs_with_location.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print("위치 정보 수집 완료!")
print(f"결과 데이터가 {output_path}에 저장되었습니다.")


검색 결과 없음: 아이티지오(주)
검색 결과 없음: 유한책임회사 굿모닝대양
검색 결과 없음: 씽씽준마지하철택배
검색 결과 없음: 파이라이트 빌딩
검색 결과 없음: 방배 임광1.2차 아파트 
검색 결과 없음: 딜리버리 유앤미
검색 결과 없음:  주식회사라포르 
검색 결과 없음: 대한불교보문종
검색 결과 없음: 도봉시니어클럽_맘편한지하철택배
검색 결과 없음: 줌마지하철택배
위치 정보 수집 완료!
결과 데이터가 ../data/cleaned_jobs_with_location.csv에 저장되었습니다.


In [5]:
df

,구인상태,업체명,직종,채용인원,근무시간,근무지역,급여,성별,연령,지원방법,직무내용,등록일,주소,위도,경도
1,구인중,(주)사람코퍼레이션,빌딩 경비원,채용인원1명,근무시간격일 7:00~익일07:00 (휴계 13시간),종로구 수송동,월급 177만원,남,~70세 까지,이메일지원 leekalver@naver.com,NaN,2025-01-20,서울 성동구 성수동2가 315-61,37.543551,127.055614
2,구인중,주식회사유케이개발,아파트 청소원,채용인원1명,근무시간평일 09~16(휴게 1) 격주 토 09~12,강남구 세곡동,월급 166만원,여,무관,전화 후 방문 2010-9466-1323,NaN,2025-01-20,서울 강북구 수유동 576-42,37.648100,127.009432
5,구인중,이문삼익아파트,아파트 청소원,채용인원1명,근무시간주 5일 09:00~15:00,동대문구 이문동,월급 117만원,여,무관,전화 후 방문 010-4843-7710,NaN,2025-01-17,서울 동대문구 이문동 73,37.595922,127.067556
8,구인중,신명스카이뷰 멤버스 오피스텔,오피스텔 경비원,채용인원1명,근무시간격일 07:00~익일07:00,서초구 서초2동,194만원,남,무관,전화 후 방문 010-5698-5113,NaN,2025-01-17,서울 서초구 서초동 1338-25,37.490593,127.029610
9,구인중,주식회사에프터클린,빌딩 경비원,채용인원1명,"근무시간격일, 07:00~22:00",종로구 신문로2가,월급 220만원,남,무관,팩스지원 0505-304-5404,NaN,2025-01-17,서울 영등포구 영등포동5가 114,37.521492,126.906100
10,구인중,주식회사 푸른종합주택관리,아파트 경비원,채용인원1명,근무시간격일 06:00~익일060:00,구로구 구로동,월급 229만원,남,무관,팩스지원 02-855-9846,NaN,2025-01-15,경기 부천시 원미구 상동 533-7,37.507648,126.751348
11,구인중,현대빌딩,빌딩 관리원,채용인원1명,"근무시간격일, 07:00~22:30",강남구 청담동,월급 230만원(발렛비 개인 수익),남,무관,전화 후 방문 010-8737-7727,NaN,2025-01-15,서울 종로구 충신동 101-2,37.574710,127.003310
13,구인중,"커피빈코리아, 봉은사점",매장보조원,채용인원1명,"근무시간주5일, 내용참조",강남구 삼성동,"시급 10,030원",무관,60세 이상,*상담사 알선 02-6220-8640,NaN,2025-01-14,서울 강남구 삼성동 77-18,37.514662,127.059170
14,구인중,파라빌딩,주차관리원,채용인원1명,"근무시간격일, 07:00~익일07:00",강동구 명일동,월급 210만원,남,무관,전화 후 방문 010-5005-9185,NaN,2025-01-13,서울 강동구 명일동 48-9,37.552361,127.154833
15,구인중,카톨릭회관,건물 청소원,채용인원1명,근무시간주5일,중구 저동,"월급 2,322,050원",여,~65세까지,팩스지원 02-772-9852,NaN,2025-01-13,서울 중구 명동2가 1-18,37.564031,126.987510
